In [42]:
import requests, json
from urllib.parse import quote
import pandas as pd
import folium

In [43]:
with open('data/roadapikey.txt') as f:
    road_key = f.read()

In [44]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'

In [45]:
ac_list = '아주대학교, 동남보건대학, 수원 경기대학교, 수원 성균관대학교, 수원여자대학교, 용인 경희대학교, 수원대학교, 수원과학대학'.split(', ')
ac_list

['아주대학교',
 '동남보건대학',
 '수원 경기대학교',
 '수원 성균관대학교',
 '수원여자대학교',
 '용인 경희대학교',
 '수원대학교',
 '수원과학대학']

In [46]:
road_addr_list = []
for ac in ac_list:
    params2 = f'keyword={quote(ac)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    if result.status_code == 200:
        res = json.loads(result.text[1:-1])
        road_addr_list.append(res['results']['juso'][0]['roadAddr'])
    else:
        print(result.status_code)

In [47]:
road_addr_list

['경기도 수원시 영통구 월드컵로 199 (원천동)',
 '경기도 수원시 장안구 천천로74번길 50 (정자동)',
 '경기도 수원시 영통구 광교산로 154-42 (이의동)',
 '경기도 수원시 장안구 서부로 2066 (천천동)',
 '경기도 용인시 처인구 이동읍 백옥대로730번길 9-63',
 '경기도 용인시 기흥구 덕영대로 1732 (서천동)',
 '경기도 화성시 봉담읍 와우안길 17',
 '경기도 화성시 정남면 세자로 288']

In [48]:
df = pd.DataFrame({
    '이름': ac_list, '주소': road_addr_list
})
df

,이름,주소
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동)
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동)
2,수원 경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동)
3,수원 성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동)
4,수원여자대학교,경기도 용인시 처인구 이동읍 백옥대로730번길 9-63
5,용인 경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동)
6,수원대학교,경기도 화성시 봉담읍 와우안길 17
7,수원과학대학,경기도 화성시 정남면 세자로 288


In [49]:
df.to_csv('data/수원소재대학교.csv', index=False)
pd.read_csv('data/수원소재대학교.csv')

,이름,주소
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동)
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동)
2,수원 경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동)
3,수원 성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동)
4,수원여자대학교,경기도 용인시 처인구 이동읍 백옥대로730번길 9-63
5,용인 경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동)
6,수원대학교,경기도 화성시 봉담읍 와우안길 17
7,수원과학대학,경기도 화성시 정남면 세자로 288


In [50]:
with open('data/kakaoapikey.txt') as f:
    kakao_key = f.read()

In [51]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '경기도 수원시 영통구 월드컵로 199 (원천동)'
url = f'{base_url}?query={quote(addr)}'
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [52]:
result = requests.get(url, headers=header).json()
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
df = pd.read_csv('data/수원소재대학교.csv')

In [53]:
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [54]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,이름,주소,위도,경도
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동),37.279205,127.044142
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동),37.296938,126.987000
2,수원 경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동),37.298014,127.033117
3,수원 성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동),37.295976,126.974209
4,수원여자대학교,경기도 용인시 처인구 이동읍 백옥대로730번길 9-63,37.204963,127.216898
5,용인 경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동),37.239800,127.081199
6,수원대학교,경기도 화성시 봉담읍 와우안길 17,37.210117,126.979480
7,수원과학대학,경기도 화성시 정남면 세자로 288,37.192033,126.983927


In [55]:
df.to_csv('data/수원소재대학교2.csv', index=False)
pd.read_csv('data/수원소재대학교2.csv')

,이름,주소,위도,경도
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동),37.279205,127.044142
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동),37.296938,126.987000
2,수원 경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동),37.298014,127.033117
3,수원 성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동),37.295976,126.974209
4,수원여자대학교,경기도 용인시 처인구 이동읍 백옥대로730번길 9-63,37.204963,127.216898
5,용인 경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동),37.239800,127.081199
6,수원대학교,경기도 화성시 봉담읍 와우안길 17,37.210117,126.979480
7,수원과학대학,경기도 화성시 정남면 세자로 288,37.192033,126.983927


In [63]:
df = pd.read_csv('data/수원소재대학교2.csv')
df['color'] = ['red','green','blue','orange','purple','black','brown','grey']
df

,이름,주소,위도,경도,color
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동),37.279205,127.044142,red
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동),37.296938,126.987000,green
2,수원 경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동),37.298014,127.033117,blue
3,수원 성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동),37.295976,126.974209,orange
4,수원여자대학교,경기도 용인시 처인구 이동읍 백옥대로730번길 9-63,37.204963,127.216898,purple
5,용인 경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동),37.239800,127.081199,black
6,수원대학교,경기도 화성시 봉담읍 와우안길 17,37.210117,126.979480,brown
7,수원과학대학,경기도 화성시 정남면 세자로 288,37.192033,126.983927,grey


In [64]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=12)
for i in df.index:
    folium.Circle(        
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=df.이름[i],
        radius=300, color=df.color[i], fill=True,
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">수원 소재 주요 대학교</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map